# Image recognition

> Written by Dr Daniel Buscombe, Northern Arizona University

> Part of a series of notebooks for image recognition and classification using deep convolutional neural networks

http://nbviewer.jupyter.org/github/thsant/scipy4cv/blob/master/07._Scikit-learn.ipynb
    
https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.07-Support-Vector-Machines.ipynb

https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.08-Random-Forests.ipynb

https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.09-Principal-Component-Analysis.ipynb

https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.11-K-Means.ipynb

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import s3fs
fs = s3fs.S3FileSystem(anon=True)

In [ ]:
frames = [f for f in fs.ls('cdi-workshop/semseg_data/sandbars') if f.endswith('.JPG')]

In [ ]:
with fs.open('cdi-workshop/imrecog_data/NWPU-RESISC45/test/airplane/airplane_700.jpg', 'rb') as f:
    image = color.rgb2gray(imread(f, 'jpg'))

In [ ]:
frames = !ls data/RC*.JPG
frames

In [ ]:

from scipy.misc import imresize
from imageio import imread

In [ ]:
examples = [imresize(imread(f), .125) for f in frames]

In [ ]:
nx, ny, nz = np.shape(examples[0])
nx, ny, nz

In [ ]:
names = [f.split('/')[-1].split('_')[0] for f in frames]

In [ ]:
target=np.arange(len(frames))

In [ ]:
fig, ax = plt.subplots(8, 5)
fig.set_figheight(15)
fig.set_figwidth(15)
for i, axi in enumerate(ax.flat):
    axi.imshow(examples[i])
    axi.set(xticks=[], yticks=[],
            ylabel=names[target[i]])

In [ ]:
examples = [imresize(imread(f), (nx,ny,nz)).reshape(-1, 1) for f in frames]

In [ ]:
np.shape(examples[0])

In [ ]:
d = {ni: indi for indi, ni in enumerate(set(names))}
target = sorted([d[ni] for ni in names])

In [ ]:
from sklearn.cross_validation import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(examples, target,
                                                random_state=42, test_size=0.25)

In [ ]:
print(len(Xtrain))
print(len(Xtest))

In [ ]:
uniq_names = np.unique(names)

In [ ]:
fig, ax = plt.subplots(3, 3)
fig.set_figheight(15)
fig.set_figwidth(15)
for i, axi in enumerate(ax.flat):
    axi.imshow(Xtest[i].reshape(nx,ny,nz))
    axi.set(xticks=[], yticks=[])
    axi.set_ylabel(uniq_names[ytest[i]],  color= 'blue')

## Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=1000)
model.fit(np.squeeze(Xtrain), ytrain)
ypred = model.predict(np.squeeze(Xtest))           

### Accuracy statistics

#### Precision

#### Recall

#### F1 score

In [ ]:
ypred, ytest
from sklearn import metrics
print(metrics.classification_report(ypred, ytest))

In [ ]:

import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.figure(figsize=(10,10))

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar(shrink=0.75)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.axis('tight')
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(ytest, ypred)
mat

In [ ]:
plot_confusion_matrix(mat, classes=uniq_names,
                      title='Confusion matrix, without normalization')

In [ ]:
# Plot normalized confusion matrix
plot_confusion_matrix(mat, classes=uniq_names, normalize=True,
                      title='Normalized confusion matrix')

## Cross-validation

we hold back some subset of the data from the training of the model, and then use this holdout set to check the model performance

a sequence of fits where each subset of the data is used both as a training set and as a validation set.

In [ ]:
Here we do two validation trials, alternately using each subset of the data as a holdout set. Using the split data from before, we could implement it like this:

In [ ]:
y2_model = model.fit(np.squeeze(Xtrain), ytrain).predict(np.squeeze(Xtest))

In [ ]:
y1_model = model.fit(np.squeeze(Xtest), ytest).predict(np.squeeze(Xtrain))

In [ ]:
metrics.accuracy_score(ytrain, y1_model), metrics.accuracy_score(ytest, y2_model)

What comes out are two accuracy scores, which we could combine (by, say, taking the mean) to get a better measure of the global model performance. This particular form of cross-validation is a two-fold cross-validation—that is, one in which we have split the data into two sets and used each in turn as a validation set.

We could expand on this idea to use even more trials, and more folds in the data

Here we split the data into five groups, and use each of them in turn to evaluate the model fit on the other 4/5 of the data. This would be rather tedious to do by hand, and so we can use Scikit-Learn's cross_val_score convenience routine to do it succinctly:

In [ ]:
from sklearn.cross_validation import cross_val_score

X = np.vstack((np.squeeze(Xtrain), np.squeeze(Xtest)))
print(np.shape(X))
y = np.hstack((ytrain, ytest))
print(np.shape(y))

In [ ]:
cross_val_score(model, X, y, cv=5)

Repeating the validation across different subsets of the data gives us an even better idea of the performance of the algorithm.

## Support vector machine

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA 
from sklearn.pipeline import make_pipeline

pca = PCA(svd_solver='randomized', n_components=10, whiten=True, random_state=42)
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)

In [ ]:
from sklearn.grid_search import GridSearchCV

In [ ]:
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid)

In [ ]:
%time grid.fit(np.squeeze(Xtrain), ytrain)

In [ ]:
print(grid.best_params_)

In [ ]:
model = grid.best_estimator_
yfit = model.predict(np.squeeze(Xtest))

yfit

In [ ]:
fig, ax = plt.subplots(4,4)
fig.set_figheight(15)
fig.set_figwidth(15)
for i, axi in enumerate(ax.flat):
    axi.imshow(Xtest[i].reshape(nx,ny,nz))
    axi.set(xticks=[], yticks=[])
    axi.set_ylabel(uniq_names[yfit[i]].split()[-1],
                   color='black' if yfit[i] == ytest[i] else 'red')
fig.suptitle('Predicted Names; Incorrect Labels in Red', size=14);

In [ ]:
print(metrics.classification_report(ytest, ypred, target_names=uniq_names))

In [ ]:
mat = confusion_matrix(ytest, yfit)

In [ ]:
plot_confusion_matrix(mat, classes=uniq_names,
                      title='Confusion matrix, without normalization')

In [ ]:
# Plot normalized confusion matrix
plot_confusion_matrix(mat, classes=uniq_names, normalize=True,
                      title='Normalized confusion matrix')


## Finding airplanes with HOG

Using HOG features, we can build up a simple image recognition algorithm with any Scikit-Learn estimator; here we will use a support vector machine.

The steps are as follows:

* Obtain a set of image thumbnails of airplanes to constitute "positive" training samples.
* Obtain a set of image thumbnails of non-airplanes to constitute "negative" training samples.
* Extract HOG features from these training samples.
* Train a SVM classifier on these samples.
* For an "unknown" image, pass a sliding window across the image, using the model to evaluate whether that window contains an airplane or not.

### Positive training samples

In [ ]:
from skimage import feature, color

In [ ]:
import s3fs
fs = s3fs.S3FileSystem(anon=True)

In [ ]:
names = [f for f in fs.ls('cdi-workshop/imrecog_data/NWPU-RESISC45/train/airplane') if f.endswith('.jpg')]

positive_patches = []
for name in names:
    with fs.open(name, 'rb') as f:
        positive_patches.append(color.rgb2gray(imread(f, 'jpg')))

In [ ]:
np.shape(positive_patches)

We have 350 airplane images to train on

### Negative training samples

In [ ]:
fs.ls('cdi-workshop/imrecog_data/NWPU-RESISC45/test/')

In [ ]:
names1 = [f for f in fs.ls('cdi-workshop/imrecog_data/NWPU-RESISC45/test/baseball_diamond') if f.endswith('.jpg')]
names2 = [f for f in fs.ls('cdi-workshop/imrecog_data/NWPU-RESISC45/test/airport') if f.endswith('.jpg')]
names = names1 + names2
len(names)

In [ ]:
negative_patches = []
for name in names:
    with fs.open(name, 'rb') as f:
        negative_patches.append(color.rgb2gray(imread(f, 'jpg')))

Let's take a look at a few of them to get an idea of what they look like:

In [ ]:
fig, ax = plt.subplots(5, 10)
fig.set_figheight(15)
fig.set_figwidth(15)
for i, axi in enumerate(ax.flat):
    axi.imshow(negative_patches[i], cmap='gray')
    axi.axis('off')

### Combine and extract HOG features

Now that we have these positive samples and negative samples, we can combine them and compute HOG features. This step takes a little while, because the HOG features involve a nontrivial computation for each image:

In [ ]:
from itertools import chain
X_train = np.array([feature.hog(im)
                    for im in chain(positive_patches,
                                    negative_patches)])

In [ ]:
nx, ny = np.shape(X_train)
y_train = np.zeros(nx)
y_train[:len(positive_patches)] = 1

In [ ]:
X_train.shape

We are left with 999 training samples in 72,900 dimensions

### Training a support vector machine

First, let's use a simple Gaussian naive Bayes to get a quick baseline:

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import cross_val_score

cross_val_score(GaussianNB(), X_train, y_train)

Accuracy is 60%

For such a high-dimensional binary classification task, a Linear support vector machine is a good choice. We will use Scikit-Learn's LinearSVC, because in comparison to SVC it often has better scaling for large number of samples.

Let's try the support vector machine, with a grid search over a few choices of the C parameter:

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.grid_search import GridSearchCV
grid = GridSearchCV(LinearSVC(), {'C': [0.125, 0.25, 0.5, 1.0]})
grid.fit(X_train, y_train)
grid.best_score_

Accuracy is 66%

In [ ]:
grid.best_params_

In [ ]:
model = grid.best_estimator_
model.fit(X_train, y_train)

### Finding airplanes in new images

In [ ]:
names = [f for f in fs.ls('cdi-workshop/imrecog_data/NWPU-RESISC45/test/airplane') if f.endswith('.jpg')]

We will use one image and run a sliding window over it and evaluate each patch:

In [ ]:
from skimage import transform
with fs.open(names[20], 'rb') as f:
    test_image = imread(f, 'jpg')
test_image = color.rgb2gray(test_image)
test_image = transform.rescale(test_image,2)
plt.imshow(test_image, cmap='gray')

Next, let's create a window that iterates over patches of this image, and compute HOG features for each patch:

In [ ]:
def sliding_window(img, patch_size=positive_patches[0].shape,
                   istep=2, jstep=2, scale=1.0):
    Ni, Nj = (int(scale * s) for s in patch_size)
    for i in range(0, img.shape[0] - Ni, istep):
        for j in range(0, img.shape[1] - Ni, jstep):
            patch = img[i:i + Ni, j:j + Nj]
            if scale != 1:
                patch = transform.resize(patch, patch_size)
            yield (i, j), patch

In [ ]:
indices, patches = zip(*sliding_window(test_image, istep=32, jstep=32))
#indices = indices[:100]
#patches = patches[:100]
len(patches)

In [ ]:
plt.imshow(patches[10], cmap='gray')

In [ ]:
patches_hog = np.array([feature.hog(patch) for patch in patches])
patches_hog.shape

Finally, we can take these HOG-featured patches and use our model to evaluate whether each patch contains an airplane:

In [ ]:
labels = model.predict(patches_hog)
labels.sum()

We see that out of all the patches, we have found a few detections. Let's use the information we have about these patches to show where they lie on our test image, drawing them as rectangles:

In [ ]:
fig, ax = plt.subplots()
ax.imshow(test_image, cmap='gray')
ax.axis('off')

Ni, Nj = positive_patches[0].shape
indices = np.array(indices)

for i, j in indices[labels == 1]:
    ax.add_patch(plt.Rectangle((j, i), Nj, Ni, edgecolor='red',
                               alpha=0.3, lw=2, facecolor='none'))

We can view this in a different way by creating a "heatmap" of identification

In [ ]:
heatmap = np.zeros(np.shape(test_image))

In [ ]:
for i, j in indices[labels == 1]:
    heatmap[i:i+Ni, j:j+Nj] = heatmap[i:i+Ni, j:j+Nj]+1

In [ ]:
fig, ax = plt.subplots()
fig.set_figheight(10)
fig.set_figwidth(10)
ax.imshow(test_image, cmap='gray')
plt.imshow(heatmap>6, alpha=0.5)
ax.axis('off')

Let's test it on some "non-airplane" imagery

In [ ]:
names = [f for f in fs.ls('cdi-workshop/imrecog_data/NWPU-RESISC45/test/baseball_diamond') if f.endswith('.jpg')]

In [ ]:
with fs.open(names[10], 'rb') as f:
    test_image = imread(f, 'jpg')
test_image = color.rgb2gray(test_image)
test_image = transform.rescale(test_image,2)
plt.imshow(test_image, cmap='gray')

In [ ]:
indices, patches = zip(*sliding_window(test_image, istep=32, jstep=32))
patches_hog = np.array([feature.hog(patch) for patch in patches])

In [ ]:
labels = model.predict(patches_hog)
labels.sum()

In [ ]:
heatmap = np.zeros(np.shape(test_image))

Ni, Nj = positive_patches[0].shape
indices = np.array(indices)

for i, j in indices[labels == 1]:
    heatmap[i:i+Ni, j:j+Nj] = heatmap[i:i+Ni, j:j+Nj]+1

In [ ]:
fig, ax = plt.subplots()
fig.set_figheight(10)
fig.set_figwidth(10)
ax.imshow(test_image, cmap='gray')
plt.imshow(heatmap, alpha=0.5)
plt.colorbar()
ax.axis('off')